A custom `jinja2.Environment().finalize` method that uses the `get_ipython().display_formatter` to format strings in `jinja2.Template`s.
This approach will allow rich display objects to be embedded in a `jinja2.Template` with less
syntax

In [1]:
    import jinja2, IPython, toolz.curried as toolz, pandas

`ipython_formatter` is used to return a `str` representation from an `IPython` mimebundle for an `object`.

In [2]:
    ip = get_ipython()

In [3]:
    def ipython_formatter(object):
        bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in toolz.pipe(mimes, toolz.map(toolz.second), toolz.concat):
            if type == 'text/plain': return object
            if type in bundle: return bundle[type]

Our template should render the same mimetypes in Jupyter notebook / Jupyter lab. To do so, we will do a little research to transcribe the 
[default types for Jupyterlab & their priority](https://github.com/jupyterlab/jupyterlab/blob/17a165ce532a1d3d00def6189d54244e95fff612/packages/rendermime/src/factories.ts).
These are the mimetypes that `jinja2` will be aware of with the same priorities. 

In [4]:
    mimes = toolz.pipe("""
    50 text/html
    90 image/bmp image/png image/jpeg image/gif'
    70 text/latex
    60 text/markdown
    80 image/svg+xml
    120 text/plain
    110 text/javascript application/javascript 
    """, str.strip, str.splitlines, toolz.map(toolz.compose(toolz.juxt(toolz.compose(int, next), list), iter, str.split)), list)

> ⚠ In the most naive implementation, not all of the outputs have properly formatted outputs.  Hopefully we add these later.

In [5]:
     show = toolz.compose(IPython.display.HTML, toolz.partial(__import__('mistune').markdown, escape=False))

In [6]:
    df = pandas.util.testing.makeDataFrame()

In [7]:
    show(jinja2.Environment().from_string("""
    This is a 😭😡💩💔 `pandas.DataFrame`
    {{df.sample(2)}}""").render(**globals()))

We can improve on this method by invoked the `pandas.DataFrame.to_html` method.

In [8]:
    show(jinja2.Environment().from_string("""
    This is a 😸😊 `pandas.DataFrame`
    {{df.sample(2).to_html()}}""").render(**globals()))

,A,B,C,D
gRmdbFSEFv,-0.481935,1.898936,-0.604074,0.921218
FHsBNHQcZc,-0.052223,1.146123,0.504821,0.711055


But it seems like needless syntax to keep track `pandas.DataFrame.to_html` while formattting a document.  Adding the __finalize__ parameter to the `jinja2.Environment` we can do more with less syntax our template.

In [9]:
    show(jinja2.Environment(finalize=ipython_formatter).from_string("""
    This is an 🚀🏆🥇🔥💯 `pandas.DataFrame`
    {{df.sample(2)}}""").render(**globals()))

,A,B,C,D
dZVMUwRDWy,0.390730,-0.308635,-0.434701,-0.660708
5l1bwlZktE,0.627521,-0.056539,0.098786,-1.011541


## Extending

Earlier we made [some custom display formatters](http://nbviewer.jupyter.org/github/deathbeds/deathbeds.github.io/tree/master/deathbeds/).  If we load in this 
extension we may use simpler syntaxes for things like `graphviz`.

    import graphviz

In [10]:
    %reload_ext deathbeds.__Custom_display_formatting
    IPython.display.HTML(jinja2.Environment(finalize=ipython_formatter).from_string("""
    Let's talk about graphs that are just
    strings wrapped in template delimiters {{"graph {rankdir=LR A--B--C--D--A}"}}""").render(**globals()))